In [ ]:
# pip install tqdm nltk gensim razdel pymorphy2

In [62]:
import multiprocessing as mpr
import os
import re
import string
from copy import copy
from typing import Iterable, Union, List

import pymorphy2
from gensim.models import FastText
from nltk.corpus import stopwords
from razdel import sentenize, tokenize
from tqdm import tqdm

Тут взяты кусочки из иного проекта, я попытался где-то упростить, но в таком виде работать будет хорошо. И потом еще и проще в сервис обернуть

In [46]:
N_CORES = mpr.cpu_count()

fasttext_lowercase_train_config = {
    "language": "russian",
    "remove_stopwords": True,
    "remove_numbers": True,
    "remove_punctuations": True,
    "remove_extra_spaces": True,
    "lowercase": True,
    "lemmatization": True,
    # model training config
    "model": FastText,
    "epochs": 10,
    "model_params": {
        "vector_size": 100,
        "window": 5,
        "min_count": 1,
        "workers": N_CORES - 1
    }
}


In [34]:
class TextPreprocessor:
    """
    Class for text preprocessing
    """

    # config key-names
    REMOVE_PUNCTUATIONS = "remove_punctuations"
    REMOVE_NUMBERS = "remove_numbers"
    REMOVE_STOPWORDS = "remove_stopwords"
    CUSTOM_STOPWORDS = "custom_stopwords"
    REMOVE_EXTRA_SPACES = "remove_extra_spaces"
    LOWERCASE = "lowercase"
    LEMMATIZATION = "lemmatization"
    YO_REPLACEMENT = "yo_replacement"
    ONLY_LETTERS = "only_letters"

    def __init__(self, config: dict):
        self.language = "russian"
        
        self.no_stopwords = config.get(self.REMOVE_STOPWORDS, False)
        self.custom_stopwords = config.get(self.CUSTOM_STOPWORDS, list())
        self.no_numbers = config.get(self.REMOVE_NUMBERS, True)
        self.no_punctuations = config.get(self.REMOVE_PUNCTUATIONS, True)
        self.no_extra_spaces = config.get(self.REMOVE_EXTRA_SPACES, True)
        self.lower_text = config.get(self.LOWERCASE, True)
        self.lemmatize = config.get(self.LEMMATIZATION, False)
        self.yo_replacement = config.get(self.YO_REPLACEMENT, True)
        self.only_letters = config.get(self.ONLY_LETTERS, False)

        self.nlp_model = pymorphy2.MorphAnalyzer()

        if self.no_stopwords:
            self.stopwords = set(stopwords.words(self.language) + self.custom_stopwords)

    @staticmethod
    def remove_extra_spaces(text: Union[str, List[str]]) -> Union[str, List[str]]:
        def _remove_extra_spaces(string):
            return re.sub(r"\s{2,}", " ", string).strip()

        if isinstance(text, str):
            return _remove_extra_spaces(text)
        if isinstance(text, Iterable):
            return [_remove_extra_spaces(token) for token in text]
        raise TypeError(f"Type {type(text)} is not supported.")

    @staticmethod
    def remove_empty_tokens(text: Union[str, List[str]]):
        if isinstance(text, str):
            return text

        return [
            token
            for token in text
            if re.sub(r"\s+", " ", token, re.IGNORECASE).strip() != ""
        ]

    @staticmethod
    def remove_numbers(text: Union[str, List[str]]) -> Union[str, List[str]]:
        def _remove_numbers(string):
            return re.sub(r"\d+\w*", "", string)

        if isinstance(text, str):
            return _remove_numbers(text)
        if isinstance(text, Iterable):
            return [_remove_numbers(token) for token in text]
        raise TypeError(f"Type {type(text)} is not supported.")

    @staticmethod
    def remove_punctuations(text: Union[str, List[str]]) -> Union[str, List[str]]:
        def _remove_punctuations(str_text):
            return re.sub(
                "[%s]" % re.escape(string.punctuation + "№«»’·⋅…"), " ", str_text
            )

        if isinstance(text, str):
            return _remove_punctuations(text)
        if isinstance(text, Iterable):
            return [_remove_punctuations(token) for token in text]
        raise TypeError(f"Type {type(text)} is not supported.")

    @staticmethod
    def make_text_lower(text: Union[str, Iterable[str]]) -> Union[str, Iterable[str]]:
        if isinstance(text, str):
            return text.lower()
        if isinstance(text, Iterable):
            return [token.lower() for token in text]
        raise TypeError(f"Type {type(text)} is not supported.")

    @staticmethod
    def replace_yo(text: Union[str, Iterable[str]]) -> Union[str, Iterable[str]]:
        if isinstance(text, str):
            return text.replace("ё", "е").replace("Ё", "Е")
        if isinstance(text, Iterable):
            return [token.replace("ё", "е").replace("Ё", "Е") for token in text]
        raise TypeError(f"Type {type(text)} is not supported.")

    def remove_stopwords(
        self, text: Union[str, Iterable[str]]
    ) -> Union[str, Iterable[str]]:
        raw_text = copy(text)
        if isinstance(text, str):
            raw_text = text.split()

        try:
            text_without_stopwords = []
            for word in raw_text:
                word = word.lower()
                if word not in self.stopwords and len(word) > 2:
                    text_without_stopwords.append(word)

            return (
                text_without_stopwords
                if not isinstance(text, str)
                else " ".join(text_without_stopwords)
            )

        except AttributeError:
            print(
                "To remove stopwords - configuration of 'remove_stopwords' is required!"
            )
            return text

    @staticmethod
    def remove_all_except_letters(text: Union[str, Iterable[str]]):
        if isinstance(text, str):
            return re.sub("[^A-Za-zА-Яа-я]+", " ", text)
        if isinstance(text, Iterable):
            return [re.sub("[^A-Za-zА-Яа-я]+", " ", token) for token in text]
        raise TypeError(f"Type {type(text)} is not supported.")

    def text_cleaning(
        self, text: Union[str, Iterable[str]]
    ) -> Union[str, Iterable[str]]:
        assert isinstance(text, str) or isinstance(
            text, Iterable
        ), f"{'Input text must be string or iterable, not'} {type(text)}"

        text = self.make_text_lower(text) if self.lower_text else text
        text = self.replace_yo(text) if self.yo_replacement else text
        text = self.remove_numbers(text) if self.no_numbers else text
        text = self.remove_punctuations(text) if self.no_punctuations else text
        text = self.remove_all_except_letters(text) if self.only_letters else text
        text = self.remove_extra_spaces(text) if self.no_extra_spaces else text
        text = self.remove_empty_tokens(text)
        text = self.remove_stopwords(text) if self.no_stopwords else text
        text = self.text_lemmatization(text) if self.lemmatize else text

        return text

    def _russian_lemmatization(self, text: Union[str, Iterable[str]]) -> Iterable[str]:
        def lemmatize(word):
            return self.nlp_model.parse(word)[0].normal_form

        if isinstance(text, str):
            text = text.split()

        return [lemmatize(word) for word in text]

    def _english_lemmatization(self, text: Union[str, Iterable[str]]) -> Iterable[str]:
        if not isinstance(text, str):
            text = " ".join(text)

        return [token.lemma_ for token in self.nlp_model(text)]

    def text_lemmatization(
        self, text: Union[str, Iterable[str]]
    ) -> Union[str, Iterable[str]]:
        if self.language == "russian":
            lemmas = self._russian_lemmatization(text=text)
        elif self.language == "english":
            lemmas = self._english_lemmatization(text=text)
        else:
            raise NotImplementedError("Only russian and english are available.")

        return " ".join(lemmas) if isinstance(text, str) else lemmas

Вот как примерно оно работает

In [35]:
PREPROCESSOR = TextPreprocessor(fasttext_lowercase_train_config)

In [36]:
PREPROCESSOR.text_cleaning("Черная металлургия всегда была и останется основой всей металлургии.")

'чёрный металлургия остаться основа весь металлургия'

In [42]:
article = """
Черная металлургия всегда была и останется основой всей металлургии. Железо, которое является готовой продукцией отрасли, известное нам как чугун, сталь и прокат из нее навсегда останутся массовым материалом потребления во всем мировом хозяйстве. Металл давно вытеснил из строительной индустрии древесину, соперничая лишь с цементом, иногда объединяясь с ним в союз – железобетон. Конечно, строительный мир не стоит на одном месте, и сейчас активно внедряется в строительство такие материалы, как полимер и керамика. Однако, пока что, железо не собирается сдавать свои позиции и держится на торговом рынке лидером продаж.
Говоря о металлургии XXI века, стоит отметить то, что до начала XX века она была очень слабо развита. Основными странами-поставщиками металла на рынок в начале XX века являлись США, Великобритания, Германия и Бельгия. Эти страны поставляли на рынок металлов более 83% от всего товара на рынке. Со временем, после Второй мировой войны, началось активное освоение металлургического комплекса рядом развитых и развивающихся стран. В последнее время мы отчетливо видим тенденция перемещения производства черных металлов из развитых стран в развивающиеся.
География черной металлургииЧерные металлы применяются во многих областях промышленности, но больше всего особым спросом они пользуются в машиностроительной промышленности. Металлы сами по себе предоставляют возможности для своего эффективного использования, при этом расширяя их, в частности благодаря покрытию стальных изделий полимерными пленками и металлами (цинком и оловом), что повышает устойчивость их иммунитета к коррозии.
Черная металлургия сильно зависит от многих других отраслей народного хозяйства. Ее сырьевая база — продукция горнодобывающей промышленности (железная руда, известняки, огнеупоры), топливной (коксующийся уголь, природный газ) и электроэнергетика.
Кроме этого, цветная металлургия – смежная черной металлургии, обеспечивает ее поставкой легируюших компонентов необходимых для разнообразных сплавов. Народное хозяйство является основным инструментом получения лома и других отходов для их вторичного использования в металлургических переделах.
ПО технологическому процессу производства черная металлургия имеет тесные связи с некоторыми разновидностями химического производства: коксованием угля, использованием кислорода и ряда инертных газов в процессах плавки материалов и т.д. С транспортной инфраструктурой черную металлургию связывает использование больших объемов сырья, получения готовой продукции и полуфабрикатов.
Производство черных металлов, их добыча и переработка сырья составляют большую опасность для экологии. Выброс газов в атмосферу и загрязнение водоемов – это еще не верхушка айсберга. Большим ударом для экосистемы является наличие отходов от производства, причем отходы невозможно утилизировать. Так самыми вредными отходами являются: канцерогены коксохимического процесса, доменные выбросы, газы и пыль при агломерировании руды, конвертерного и других плавильных агрегатов, шлаки всех металлургических переделов.
В процессе производств используется большое количество воды, температура после ее участия в металлургических процессах изменяется и таким образом, попадание в эту воду отходов промышленности приводит к химическому и температурному нарушению режима естественных источников водоснабжения.
На сегодняшний день, с экономической точки зрения, черная металлургия является одной из малодоходных отраслей обрабатывающей промышленности. Больших вложений капиталов требует развитие всех производств черной металлургии, что обусловлено технологией выпускаемой продукции – большими объемами производства, дороговизной оборудования, финансовыми затратами на его амортизацию и созданием огромной сети заводской инфраструктуры, отвечающей современным требованиям. Затраты предприятий на обеспечение экологической чистоты работы металлургического предприятия могут достигать до 20 % общих капиталовложений.
"""

Вот это основная строчка, которая преобразует предложения.
В итоге должен получится список списков (```list[list]```). Это структура, которая воспринимается fasttext.

In [48]:
sentences = [[x.text for x in tokenize(PREPROCESSOR.text_cleaning(sentence.text))] for sentence in sentenize(article)]

Статья для примера, тут все будет зависеть от вашей реализации: я бы, наверное, предложил что-то вроде:

In [49]:
articles = ["""
Черная металлургия всегда была и останется основой всей металлургии. Железо, которое является готовой продукцией отрасли, известное нам как чугун, сталь и прокат из нее навсегда останутся массовым материалом потребления во всем мировом хозяйстве. Металл давно вытеснил из строительной индустрии древесину, соперничая лишь с цементом, иногда объединяясь с ним в союз – железобетон. Конечно, строительный мир не стоит на одном месте, и сейчас активно внедряется в строительство такие материалы, как полимер и керамика. Однако, пока что, железо не собирается сдавать свои позиции и держится на торговом рынке лидером продаж.
Говоря о металлургии XXI века, стоит отметить то, что до начала XX века она была очень слабо развита. Основными странами-поставщиками металла на рынок в начале XX века являлись США, Великобритания, Германия и Бельгия. Эти страны поставляли на рынок металлов более 83% от всего товара на рынке. Со временем, после Второй мировой войны, началось активное освоение металлургического комплекса рядом развитых и развивающихся стран. В последнее время мы отчетливо видим тенденция перемещения производства черных металлов из развитых стран в развивающиеся.""",
            
"""География черной металлургииЧерные металлы применяются во многих областях промышленности, но больше всего особым спросом они пользуются в машиностроительной промышленности. Металлы сами по себе предоставляют возможности для своего эффективного использования, при этом расширяя их, в частности благодаря покрытию стальных изделий полимерными пленками и металлами (цинком и оловом), что повышает устойчивость их иммунитета к коррозии.
Черная металлургия сильно зависит от многих других отраслей народного хозяйства. Ее сырьевая база — продукция горнодобывающей промышленности (железная руда, известняки, огнеупоры), топливной (коксующийся уголь, природный газ) и электроэнергетика.
Кроме этого, цветная металлургия – смежная черной металлургии, обеспечивает ее поставкой легируюших компонентов необходимых для разнообразных сплавов. Народное хозяйство является основным инструментом получения лома и других отходов для их вторичного использования в металлургических переделах.
ПО технологическому процессу производства черная металлургия имеет тесные связи с некоторыми разновидностями химического производства: коксованием угля, использованием кислорода и ряда инертных газов в процессах плавки материалов и т.д. С транспортной инфраструктурой черную металлургию связывает использование больших объемов сырья, получения готовой продукции и полуфабрикатов.
Производство черных металлов, их добыча и переработка сырья составляют большую опасность для экологии. Выброс газов в атмосферу и загрязнение водоемов – это еще не верхушка айсберга. Большим ударом для экосистемы является наличие отходов от производства, причем отходы невозможно утилизировать. Так самыми вредными отходами являются: канцерогены коксохимического процесса, доменные выбросы, газы и пыль при агломерировании руды, конвертерного и других плавильных агрегатов, шлаки всех металлургических переделов.
""",
"""В процессе производств используется большое количество воды, температура после ее участия в металлургических процессах изменяется и таким образом, попадание в эту воду отходов промышленности приводит к химическому и температурному нарушению режима естественных источников водоснабжения.
На сегодняшний день, с экономической точки зрения, черная металлургия является одной из малодоходных отраслей обрабатывающей промышленности. Больших вложений капиталов требует развитие всех производств черной металлургии, что обусловлено технологией выпускаемой продукции – большими объемами производства, дороговизной оборудования, финансовыми затратами на его амортизацию и созданием огромной сети заводской инфраструктуры, отвечающей современным требованиям. Затраты предприятий на обеспечение экологической чистоты работы металлургического предприятия могут достигать до 20 % общих капиталовложений.
"""]

sentences = []
for article in tqdm(articles, desc=f"{'Getting sentences'}"):
    sentences.extend(
        [[x.text for x in tokenize(PREPROCESSOR.text_cleaning(sentence.text))] for sentence in sentenize(article)]
    )

Getting sentences: 100%|██████████| 3/3 [00:00<00:00, 68.19it/s]


Тут определяем модель, по сути это аналог вот такой структуры:
```python
model = FastText(vector_size=100, window=5, min_count=1)
```

По документации можно тут посмотреть: https://radimrehurek.com/gensim/models/fasttext.html

In [52]:
model = fasttext_lowercase_train_config["model"](**fasttext_lowercase_train_config["model_params"])

Строим словарь

In [53]:
model.build_vocab(corpus_iterable=sentences)

Учим модель. Проще учить по эпохам: так интереснее контролировать процесс

In [54]:
for epoch in range(fasttext_lowercase_train_config.get("epochs")):
    model.train(
        corpus_iterable=tqdm(sentences, desc="Training model"),
        total_examples=len(sentences),
        epochs=1,
    )


Training model: 100%|██████████| 26/26 [00:00<00:00, 3249.07it/s]

Training model: 100%|██████████| 26/26 [00:00<00:00, 3714.93it/s]

Training model: 100%|██████████| 26/26 [00:00<00:00, 3249.75it/s]

Training model: 100%|██████████| 26/26 [00:00<00:00, 4330.89it/s]

Training model: 100%|██████████| 26/26 [00:00<00:00, 3714.18it/s]

Training model: 100%|██████████| 26/26 [00:00<00:00, 3660.07it/s]

Training model: 100%|██████████| 26/26 [00:00<00:00, 3716.20it/s]

Training model: 100%|██████████| 26/26 [00:00<00:00, 4333.48it/s]

Training model: 100%|██████████| 26/26 [00:00<00:00, 3249.94it/s]

Training model: 100%|██████████| 26/26 [00:00<00:00, 3251.11it/s]


Это словарь, который у вас соберется в процессе

In [55]:
vocab = model.wv.key_to_index
print(vocab)

{'чёрный': 0, 'металлургия': 1, 'металл': 2, 'производство': 3, 'являться': 4, 'металлургический': 5, 'промышленность': 6, 'больший': 7, 'процесс': 8, 'отход': 9, 'страна': 10, 'рынок': 11, 'использование': 12, 'продукция': 13, 'хозяйство': 14, 'материал': 15, 'век': 16, 'отрасль': 17, 'газ': 18, 'другой': 19, 'начало': 20, 'выброс': 21, 'время': 22, 'сырьё': 23, 'свой': 24, 'вода': 25, 'объём': 26, 'химический': 27, 'инфраструктура': 28, 'развитой': 29, 'развивающийся': 30, 'один': 31, 'передел': 32, 'получение': 33, 'многий': 34, 'уголь': 35, 'руда': 36, 'народный': 37, 'такой': 38, 'основный': 39, 'мировой': 40, 'строительный': 41, 'предприятие': 42, 'железо': 43, 'затрата': 44, 'весь': 45, 'готовый': 46, 'остаться': 47, 'стоить': 48, 'навсегда': 49, 'металлургиичерный': 50, 'область': 51, 'известный': 52, 'мы': 53, 'применяться': 54, 'чугун': 55, 'массовый': 56, 'последний': 57, 'сталь': 58, 'отчётливо': 59, 'география': 60, 'особый': 61, 'перемещение': 62, 'тенденция': 63, 'видеть

Потом модель нужно сохранить

In [64]:
PATH_FOR_SAVING = "models"
MODEL_NAME = "fasttext_sample"

model_path = os.path.join(
    PATH_FOR_SAVING,
    MODEL_NAME + ".model",
)
model.save(model_path)

Ну и потом загрузить

In [66]:
model = FastText.load(model_path)

Как искать синонимы

In [67]:
model.wv.most_similar("плавка")

[('поставка', 0.482550710439682),
 ('пока', 0.41134727001190186),
 ('экономический', 0.39740726351737976),
 ('огромный', 0.38887518644332886),
 ('химический', 0.3745107352733612),
 ('коксохимический', 0.3730074465274811),
 ('позиция', 0.3715028762817383),
 ('затрата', 0.3702733516693115),
 ('область', 0.3686293363571167),
 ('инфраструктура', 0.3623194396495819)]

Тут куча проблем из-за того, что маленький (почти ничтожный) корпус. Но это и видно по цифрам. Метод выдает синоним и косинусную меру (cosine similarity) близости.

In [68]:
model.wv.most_similar("капитал")

[('капиталовложение', 0.4658592939376831),
 ('чистота', 0.3421204388141632),
 ('создание', 0.3293217420578003),
 ('вытеснить', 0.3263553977012634),
 ('попадание', 0.32500159740448),
 ('продажа', 0.3163697421550751),
 ('обрабатывать', 0.31310442090034485),
 ('производство', 0.31207454204559326),
 ('передел', 0.3022144138813019),
 ('сталь', 0.29675740003585815)]